任务：主 agent 接受来自用户的任务，首先确定问题是否能够回答（护栏边界内），如果在护栏内，那么调用对应的 agent 回答

In [1]:
from agents.extensions.models.litellm_model import LitellmModel
from agents import set_tracing_disabled
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = "https://api.deepseek.com/v1"
chat_model = "deepseek/deepseek-chat"

# chat_model = "mistral-small-latest"
# base_url="https://api.mistral.ai/v1"
# api_key=os.getenv('mistral_key')

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

In [2]:
from agents import Agent

# 历史
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

# 数学
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

In [4]:
from agents import GuardrailFunctionOutput, InputGuardrail, Agent, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    """定义输出数据模型"""
    is_homework: bool
    reasoning: str

# 创建护栏代理
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="""Check if the user is asking about homework.
    
    Return your response in the following JSON format:
    {
        "is_homework": true/false,
        "reasoning": "Your reasoning here"
    }
    """,
    output_type=HomeworkOutput,  # 约定输出格式
    model=llm,
)

# 异步函数
async def homework_guardrail(ctx, agent, input_data):
    # 传入护栏代理、输入数据、上下文
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    # 将结果转换为 HomeworkOutput 格式
    return GuardrailFunctionOutput(
        output_info=final_output,  # 包含结果的详细信息
        tripwire_triggered=not final_output.is_homework,  # 出发条件，如果 is_homework=False，则出发护栏
    )

In [5]:
# 基于用户问题，选择合适的 agent
# triage_agent = Agent(
#     name="Triage Agent",
#     instructions="You determine which agent to use based on the user's homework question",
#     handoffs=[history_tutor_agent, math_tutor_agent],
#     model=llm,
# )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    # 允许代理将特定任务委派给其他代理
    handoffs=[history_tutor_agent, math_tutor_agent],
    # 可以对输入到代理的内容进行验证
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
    model=llm,
)

In [6]:
from agents import Runner
from agents.exceptions import InputGuardrailTripwireTriggered

# 测试数学问题（应该通过护栏）
try:
    result = await Runner.run(triage_agent, "欧拉公式的应用？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")

print("\n" + "="*50 + "\n")

# 测试历史问题（应该通过护栏）
try:
    result = await Runner.run(triage_agent, "美国第一任总统是谁？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")

print("\n" + "="*50 + "\n")

# 测试非学术问题（应该触发护栏）
try:
    result = await Runner.run(triage_agent, "今天天气怎么样？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: DeepseekException - Failed to deserialize the JSON body into the target type: response_format: This response_format type is unavailable now at line 1 column 813

In [ ]:
from agents import Runner
import asyncio
from agents.exceptions import InputGuardrailTripwireTriggered

# async def main():
#     result = await Runner.run(triage_agent, "does drinking tee good for the body?")
#     print(result.final_output)

    
# if __name__ == "__main__":
    # asyncio.run(main())

result = await Runner.run(triage_agent, "欧拉公式的应用？")
print(result.final_output)


# try:
#     result = await Runner.run(triage_agent, "who was the first president of the united states?")
#     print(result.final_output)
# except InputGuardrailTripwireTriggered as e:
#     print("你的问题不是作业相关")

报错没有解决！

```shell
---------------------------------------------------------------------------
HTTPStatusError                           Traceback (most recent call last)
File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/llms/custom_httpx/llm_http_handler.py:111, in BaseLLMHTTPHandler._make_common_async_call(self, async_httpx_client, provider_config, api_base, headers, data, timeout, litellm_params, logging_obj, stream, signed_json_body)
    110 try:
--> 111     response = await async_httpx_client.post(
    112         url=api_base,
    113         headers=headers,
    114         data=(
    115             signed_json_body
    116             if signed_json_body is not None
    117             else json.dumps(data)
    118         ),
    119         timeout=timeout,
    120         stream=stream,
    121         logging_obj=logging_obj,
    122     )
    123 except httpx.HTTPStatusError as e:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/litellm_core_utils/logging_utils.py:135, in track_llm_api_timing.<locals>.decorator.<locals>.async_wrapper(*args, **kwargs)
    134 try:
--> 135     result = await func(*args, **kwargs)
    136     return result

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/llms/custom_httpx/http_handler.py:324, in AsyncHTTPHandler.post(self, url, data, json, params, headers, timeout, stream, logging_obj, files, content)
    322     setattr(e, "status_code", e.response.status_code)
--> 324     raise e
    325 except Exception as e:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/llms/custom_httpx/http_handler.py:280, in AsyncHTTPHandler.post(self, url, data, json, params, headers, timeout, stream, logging_obj, files, content)
    279 response = await self.client.send(req, stream=stream)
--> 280 response.raise_for_status()
    281 return response

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/httpx/_models.py:829, in Response.raise_for_status(self)
    828 message = message.format(self, error_type=error_type)
--> 829 raise HTTPStatusError(message, request=request, response=self)

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'https://api.deepseek.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422

During handling of the above exception, another exception occurred:

OpenAIError                               Traceback (most recent call last)
File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/main.py:541, in acompletion(model, messages, functions, function_call, timeout, temperature, top_p, n, stream, stream_options, stop, max_tokens, max_completion_tokens, modalities, prediction, audio, presence_penalty, frequency_penalty, logit_bias, user, response_format, seed, tools, tool_choice, parallel_tool_calls, logprobs, top_logprobs, deployment_id, reasoning_effort, base_url, api_version, api_key, model_list, extra_headers, thinking, web_search_options, **kwargs)
    540 elif asyncio.iscoroutine(init_response):
--> 541     response = await init_response
    542 else:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/llms/custom_httpx/llm_http_handler.py:238, in BaseLLMHTTPHandler.async_completion(self, custom_llm_provider, provider_config, api_base, headers, data, timeout, model, model_response, logging_obj, messages, optional_params, litellm_params, encoding, api_key, client, json_mode, signed_json_body)
    236     async_httpx_client = client
--> 238 response = await self._make_common_async_call(
    239     async_httpx_client=async_httpx_client,
    240     provider_config=provider_config,
    241     api_base=api_base,
    242     headers=headers,
    243     data=data,
    244     timeout=timeout,
    245     litellm_params=litellm_params,
    246     stream=False,
    247     logging_obj=logging_obj,
    248     signed_json_body=signed_json_body,
    249 )
    250 return provider_config.transform_response(
    251     model=model,
    252     raw_response=response,
   (...)
    261     json_mode=json_mode,
    262 )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/llms/custom_httpx/llm_http_handler.py:136, in BaseLLMHTTPHandler._make_common_async_call(self, async_httpx_client, provider_config, api_base, headers, data, timeout, litellm_params, logging_obj, stream, signed_json_body)
    135     else:
--> 136         raise self._handle_error(e=e, provider_config=provider_config)
    137 except Exception as e:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/llms/custom_httpx/llm_http_handler.py:2405, in BaseLLMHTTPHandler._handle_error(self, e, provider_config)
   2399     raise BaseLLMException(
   2400         status_code=status_code,
   2401         message=error_text,
   2402         headers=error_headers,
   2403     )
-> 2405 raise provider_config.get_error_class(
   2406     error_message=error_text,
   2407     status_code=status_code,
   2408     headers=error_headers,
   2409 )

OpenAIError: Failed to deserialize the JSON body into the target type: response_format: This response_format type is unavailable now at line 1 column 633

During handling of the above exception, another exception occurred:

BadRequestError                           Traceback (most recent call last)
Cell In[5], line 18
      6     print(result.final_output)
      9 # async def main():
     10 #     result = await Runner.run(triage_agent, "does drinking tee good for the body?")
     11 #     print(result.final_output)
   (...)
     14 # if __name__ == "__main__":
     15     # asyncio.run(main())
---> 18 result = await Runner.run(triage_agent, "does drinking tee good for the body?")
     19 print(result.final_output)

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:200, in Runner.run(cls, starting_agent, input, context, max_turns, hooks, run_config, previous_response_id)
    173 """Run a workflow starting at the given agent. The agent will run in a loop until a final
    174 output is generated. The loop runs like so:
    175 1. The agent is invoked with the given input.
   (...)
    197     agent. Agents may perform handoffs, so we don't know the specific type of the output.
    198 """
    199 runner = DEFAULT_AGENT_RUNNER
--> 200 return await runner.run(
    201     starting_agent,
    202     input,
    203     context=context,
    204     max_turns=max_turns,
    205     hooks=hooks,
    206     run_config=run_config,
    207     previous_response_id=previous_response_id,
    208 )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:397, in AgentRunner.run(self, starting_agent, input, **kwargs)
    392 logger.debug(
    393     f"Running agent {current_agent.name} (turn {current_turn})",
    394 )
    396 if current_turn == 1:
--> 397     input_guardrail_results, turn_result = await asyncio.gather(
    398         self._run_input_guardrails(
    399             starting_agent,
    400             starting_agent.input_guardrails
    401             + (run_config.input_guardrails or []),
    402             copy.deepcopy(input),
    403             context_wrapper,
    404         ),
    405         self._run_single_turn(
    406             agent=current_agent,
    407             all_tools=all_tools,
    408             original_input=original_input,
    409             generated_items=generated_items,
    410             hooks=hooks,
    411             context_wrapper=context_wrapper,
    412             run_config=run_config,
    413             should_run_agent_start_hooks=should_run_agent_start_hooks,
    414             tool_use_tracker=tool_use_tracker,
    415             previous_response_id=previous_response_id,
    416         ),
    417     )
    418 else:
    419     turn_result = await self._run_single_turn(
    420         agent=current_agent,
    421         all_tools=all_tools,
   (...)
    429         previous_response_id=previous_response_id,
    430     )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:997, in AgentRunner._run_input_guardrails(cls, agent, guardrails, input, context)
    994 guardrail_results = []
    996 for done in asyncio.as_completed(guardrail_tasks):
--> 997     result = await done
    998     if result.output.tripwire_triggered:
    999         # Cancel all guardrail tasks if a tripwire is triggered.
   1000         for t in guardrail_tasks:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/asyncio/tasks.py:571, in as_completed.<locals>._wait_for_one()
    568 if f is None:
    569     # Dummy value from _on_timeout().
    570     raise exceptions.TimeoutError
--> 571 return f.result()

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/_run_impl.py:892, in RunImpl.run_single_input_guardrail(cls, agent, guardrail, input, context)
    883 @classmethod
    884 async def run_single_input_guardrail(
    885     cls,
   (...)
    889     context: RunContextWrapper[TContext],
    890 ) -> InputGuardrailResult:
    891     with guardrail_span(guardrail.get_name()) as span_guardrail:
--> 892         result = await guardrail.run(agent, input, context)
    893         span_guardrail.span_data.triggered = result.output.tripwire_triggered
    894         return result

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/guardrail.py:118, in InputGuardrail.run(self, agent, input, context)
    114 output = self.guardrail_function(context, agent, input)
    115 if inspect.isawaitable(output):
    116     return InputGuardrailResult(
    117         guardrail=self,
--> 118         output=await output,
    119     )
    121 return InputGuardrailResult(
    122     guardrail=self,
    123     output=output,
    124 )

Cell In[3], line 20
     18 async def homework_guardrail(ctx, agent, input_data):
     19     # 传入护栏代理、输入数据、上下文
---> 20     result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
     21     final_output = result.final_output_as(HomeworkOutput)
     22     # 将结果转换为 HomeworkOutput 格式

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:200, in Runner.run(cls, starting_agent, input, context, max_turns, hooks, run_config, previous_response_id)
    173 """Run a workflow starting at the given agent. The agent will run in a loop until a final
    174 output is generated. The loop runs like so:
    175 1. The agent is invoked with the given input.
   (...)
    197     agent. Agents may perform handoffs, so we don't know the specific type of the output.
    198 """
    199 runner = DEFAULT_AGENT_RUNNER
--> 200 return await runner.run(
    201     starting_agent,
    202     input,
    203     context=context,
    204     max_turns=max_turns,
    205     hooks=hooks,
    206     run_config=run_config,
    207     previous_response_id=previous_response_id,
    208 )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:397, in AgentRunner.run(self, starting_agent, input, **kwargs)
    392 logger.debug(
    393     f"Running agent {current_agent.name} (turn {current_turn})",
    394 )
    396 if current_turn == 1:
--> 397     input_guardrail_results, turn_result = await asyncio.gather(
    398         self._run_input_guardrails(
    399             starting_agent,
    400             starting_agent.input_guardrails
    401             + (run_config.input_guardrails or []),
    402             copy.deepcopy(input),
    403             context_wrapper,
    404         ),
    405         self._run_single_turn(
    406             agent=current_agent,
    407             all_tools=all_tools,
    408             original_input=original_input,
    409             generated_items=generated_items,
    410             hooks=hooks,
    411             context_wrapper=context_wrapper,
    412             run_config=run_config,
    413             should_run_agent_start_hooks=should_run_agent_start_hooks,
    414             tool_use_tracker=tool_use_tracker,
    415             previous_response_id=previous_response_id,
    416         ),
    417     )
    418 else:
    419     turn_result = await self._run_single_turn(
    420         agent=current_agent,
    421         all_tools=all_tools,
   (...)
    429         previous_response_id=previous_response_id,
    430     )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:910, in AgentRunner._run_single_turn(cls, agent, all_tools, original_input, generated_items, hooks, context_wrapper, run_config, should_run_agent_start_hooks, tool_use_tracker, previous_response_id)
    907 input = ItemHelpers.input_to_new_input_list(original_input)
    908 input.extend([generated_item.to_input_item() for generated_item in generated_items])
--> 910 new_response = await cls._get_new_response(
    911     agent,
    912     system_prompt,
    913     input,
    914     output_schema,
    915     all_tools,
    916     handoffs,
    917     context_wrapper,
    918     run_config,
    919     tool_use_tracker,
    920     previous_response_id,
    921     prompt_config,
    922 )
    924 return await cls._get_single_step_result_from_response(
    925     agent=agent,
    926     original_input=original_input,
   (...)
    935     tool_use_tracker=tool_use_tracker,
    936 )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/run.py:1071, in AgentRunner._get_new_response(cls, agent, system_prompt, input, output_schema, all_tools, handoffs, context_wrapper, run_config, tool_use_tracker, previous_response_id, prompt_config)
   1068 model_settings = agent.model_settings.resolve(run_config.model_settings)
   1069 model_settings = RunImpl.maybe_reset_tool_choice(agent, tool_use_tracker, model_settings)
-> 1071 new_response = await model.get_response(
   1072     system_instructions=system_prompt,
   1073     input=input,
   1074     model_settings=model_settings,
   1075     tools=all_tools,
   1076     output_schema=output_schema,
   1077     handoffs=handoffs,
   1078     tracing=get_model_tracing_impl(
   1079         run_config.tracing_disabled, run_config.trace_include_sensitive_data
   1080     ),
   1081     previous_response_id=previous_response_id,
   1082     prompt=prompt_config,
   1083 )
   1085 context_wrapper.usage.add(new_response.usage)
   1087 return new_response

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/extensions/models/litellm_model.py:82, in LitellmModel.get_response(self, system_instructions, input, model_settings, tools, output_schema, handoffs, tracing, previous_response_id, prompt)
     64 async def get_response(
     65     self,
     66     system_instructions: str | None,
   (...)
     74     prompt: Any | None = None,
     75 ) -> ModelResponse:
     76     with generation_span(
     77         model=str(self.model),
     78         model_config=model_settings.to_json_dict()
     79         | {"base_url": str(self.base_url or ""), "model_impl": "litellm"},
     80         disabled=tracing.is_disabled(),
     81     ) as span_generation:
---> 82         response = await self._fetch_response(
     83             system_instructions,
     84             input,
     85             model_settings,
     86             tools,
     87             output_schema,
     88             handoffs,
     89             span_generation,
     90             tracing,
     91             stream=False,
     92             prompt=prompt,
     93         )
     95         assert isinstance(response.choices[0], litellm.types.utils.Choices)
     97         if _debug.DONT_LOG_MODEL_DATA:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/agents/extensions/models/litellm_model.py:301, in LitellmModel._fetch_response(self, system_instructions, input, model_settings, tools, output_schema, handoffs, span, tracing, stream, prompt)
    298 if model_settings.extra_args:
    299     extra_kwargs.update(model_settings.extra_args)
--> 301 ret = await litellm.acompletion(
    302     model=self.model,
    303     messages=converted_messages,
    304     tools=converted_tools or None,
    305     temperature=model_settings.temperature,
    306     top_p=model_settings.top_p,
    307     frequency_penalty=model_settings.frequency_penalty,
    308     presence_penalty=model_settings.presence_penalty,
    309     max_tokens=model_settings.max_tokens,
    310     tool_choice=self._remove_not_given(tool_choice),
    311     response_format=self._remove_not_given(response_format),
    312     parallel_tool_calls=parallel_tool_calls,
    313     stream=stream,
    314     stream_options=stream_options,
    315     reasoning_effort=reasoning_effort,
    316     extra_headers={**HEADERS, **(model_settings.extra_headers or {})},
    317     api_key=self.api_key,
    318     base_url=self.base_url,
    319     **extra_kwargs,
    320 )
    322 if isinstance(ret, litellm.types.utils.ModelResponse):
    323     return ret

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/utils.py:1552, in client.<locals>.wrapper_async(*args, **kwargs)
   1550 timeout = _get_wrapper_timeout(kwargs=kwargs, exception=e)
   1551 setattr(e, "timeout", timeout)
-> 1552 raise e

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/utils.py:1410, in client.<locals>.wrapper_async(*args, **kwargs)
   1407         print_verbose(f"Error while checking max token limit: {str(e)}")
   1409 # MODEL CALL
-> 1410 result = await original_function(*args, **kwargs)
   1411 end_time = datetime.datetime.now()
   1412 if _is_streaming_request(
   1413     kwargs=kwargs,
   1414     call_type=call_type,
   1415 ):

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/main.py:560, in acompletion(model, messages, functions, function_call, timeout, temperature, top_p, n, stream, stream_options, stop, max_tokens, max_completion_tokens, modalities, prediction, audio, presence_penalty, frequency_penalty, logit_bias, user, response_format, seed, tools, tool_choice, parallel_tool_calls, logprobs, top_logprobs, deployment_id, reasoning_effort, base_url, api_version, api_key, model_list, extra_headers, thinking, web_search_options, **kwargs)
    558 except Exception as e:
    559     custom_llm_provider = custom_llm_provider or "openai"
--> 560     raise exception_type(
    561         model=model,
    562         custom_llm_provider=custom_llm_provider,
    563         original_exception=e,
    564         completion_kwargs=completion_kwargs,
    565         extra_kwargs=kwargs,
    566     )

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/litellm_core_utils/exception_mapping_utils.py:2293, in exception_type(model, original_exception, custom_llm_provider, completion_kwargs, extra_kwargs)
   2291 if exception_mapping_worked:
   2292     setattr(e, "litellm_response_headers", litellm_response_headers)
-> 2293     raise e
   2294 else:
   2295     for error_type in litellm.LITELLM_EXCEPTION_TYPES:

File /devtool/anaconda3/envs/openai-agent/lib/python3.10/site-packages/litellm/litellm_core_utils/exception_mapping_utils.py:482, in exception_type(model, original_exception, custom_llm_provider, completion_kwargs, extra_kwargs)
    480 elif original_exception.status_code == 422:
    481     exception_mapping_worked = True
--> 482     raise BadRequestError(
    483         message=f"{exception_provider} - {message}",
    484         model=model,
    485         llm_provider=custom_llm_provider,
    486         response=getattr(original_exception, "response", None),
    487         litellm_debug_info=extra_information,
    488         body=getattr(original_exception, "body", None),
    489     )
    490 elif original_exception.status_code == 429:
    491     exception_mapping_worked = True

BadRequestError: litellm.BadRequestError: DeepseekException - Failed to deserialize the JSON body into the target type: response_format: This response_format type is unavailable now at line 1 column 633```